In [45]:
##Load the necessary Libraries
!pip install farasapy
import re
import string
from farasa.stemmer import FarasaStemmer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk as nltk
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
#Filtering out the warnings

import warnings

warnings.filterwarnings('ignore')

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
##Unicode
HAMZA = u'\u0621'
ALEF_MADDA = u'\u0622'
ALEF_HAMZA_ABOVE = u'\u0623'
WAW_HAMZA = u'\u0624'
ALEF_HAMZA_BELOW = u'\u0625'
YEH_HAMZA = u'\u0626'
ALEF = u'\u0627'
BEH = u'\u0628'
TEH_MARBUTA = u'\u0629'
TEH = u'\u062a'
THEH = u'\u062b'
JEEM = u'\u062c'
HAH = u'\u062d'
KHAH = u'\u062e'
DAL = u'\u062f'
THAL = u'\u0630'
REH = u'\u0631'
ZAIN = u'\u0632'
SEEN = u'\u0633'
SHEEN = u'\u0634'
SAD = u'\u0635'
DAD = u'\u0636'
TAH = u'\u0637'
ZAH = u'\u0638'
AIN = u'\u0639'
GHAIN = u'\u063a'
TATWEEL = u'\u0640'
FEH = u'\u0641'
QAF = u'\u0642'
KAF = u'\u0643'
LAM = u'\u0644'
MEEM = u'\u0645'
NOON = u'\u0646'
HEH = u'\u0647'
WAW = u'\u0648'
ALEF_MAKSURA = u'\u0649'
YEH = u'\u064a'
MADDA_ABOVE = u'\u0653'
HAMZA_ABOVE = u'\u0654'
HAMZA_BELOW = u'\u0655'
ZERO = u'\u0660'
ONE = u'\u0661'
TWO = u'\u0662'
THREE = u'\u0663'
FOUR = u'\u0664'
FIVE = u'\u0665'
SIX = u'\u0666'
SEVEN = u'\u0667'
EIGHT = u'\u0668'
NINE = u'\u0669'
PERCENT = u'\u066a'
DECIMAL = u'\u066b'
THOUSANDS = u'\u066c'
STAR = u'\u066d'
MINI_ALEF = u'\u0670'
ALEF_WASLA = u'\u0671'
FULL_STOP = u'\u06d4'
BYTE_ORDER_MARK = u'\ufeff'

In [47]:
df = pd.read_csv('arabic_news.csv')
df['label'] = df['label'].apply(lambda x: 1 if x == 'pos\t\t\t\t' else 0)
df.head()


,text,label
0,تراجع التضخم النصف سنوي في الجزائر,1
1,صكوك من الكويت بملياري دولار,1
2,الحرب في سوريا: قافلة الإغاثة المرتقبة لن تتمك...,0
3,الأساطير المحيطة بالممثلين المجوس في مصر تتحطم...,1
4,مصر: مجلس النواب يقر حصانة قضائية لضباط كبار ب...,0


In [48]:
#1
emoji_group =u"\U0001F600-\U0001F64F", u"\U0001F300-\U0001F5FF", u"\U0001F680-\U0001F6FF", u"\U0001F1E0-\U0001F1FF"  
def remove_emojis(text):
    emoji_pattern = re.compile("emoji_group", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
#2
def remove_non_arabic(text):
    return ' '.join(re.sub(u"[^\u0621-\u063A\u0640-\u0652 ]", " ", text,  flags=re.UNICODE).split())
#3
def strip_tatweel(text):
    return re.sub(u'[%s]' % TATWEEL, '', text)
#4
def remove_underscore(text):
    return ' '.join(text.split('_'))
#5
def remove_extra_spaces(text):
    return ' '.join(text.split())
#6
def remove_retweet_tag(text):
    return re.compile('\#').sub('', re.compile('rt @[a-zA-Z0-9_]+:|@[a-zA-Z0-9_]+').sub('', text).strip())
df['C_non_ar'] = df['text'].apply(lambda x: remove_non_arabic(x))
df['C_tatweel'] = df['C_non_ar'].apply(lambda x: strip_tatweel(x))
df['C__underscore'] = df['C_tatweel'].apply(lambda x: remove_underscore(x))
df['C_extra_spaces'] = df['C__underscore'].apply(lambda x: remove_extra_spaces(x))
df['C_retweet_tag'] = df['C_extra_spaces'].apply(lambda x: remove_retweet_tag(x))
df['C_emojis'] = df['C_retweet_tag'].apply(lambda x: remove_emojis(x))

df = df[['label', 'text', 'C_emojis']]
df = df.rename(columns={'text': 'old_text', 'C_emojis': 'text'})
df = df[['label', 'text']]
df.head()
  

,label,text
0,1,تراجع التضخم النصف سنوي في الجزائر
1,1,صكوك من الكويت بملياري دولار
2,0,الحرب في سوريا قافلة الإغاثة المرتقبة لن تتمكن...
3,1,الأساطير المحيطة بالممثلين المجوس في مصر تتحطم...
4,0,مصر مجلس النواب يقر حصانة قضائية لضباط كبار با...


In [49]:
#1
#punctuation
import string
string.punctuation
def remove_punct(text):
    
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

df['nopunc'] = df['text'].apply(lambda x: remove_punct(x))

#2
#tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens
df['tokenized'] = df['nopunc'].apply(lambda x: tokenize(x))

#3
#stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_Ar = nltk.corpus.stopwords.words('arabic')
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopwords_Ar]
    return text

df['nostop'] = df['tokenized'].apply(lambda x: remove_stopwords(x))

#4
#stemmer
!pip install snowballstemmer
from snowballstemmer import stemmer
ar_stem = stemmer("arabic")
import nltk
ps = nltk.PorterStemmer()
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

df['stemmed'] = df['nostop'].apply(lambda x: stemming(x))

### Create function to remove punctuation, tokenize, remove stopwords, and stem

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
     #tokens = re.split('\W+', text)
    tokens = word_tokenize(text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords_Ar])
    return text
df=df[['label','text']]
df['cleaned_text'] = df['text'].apply(lambda x: clean_text(x))

from sklearn.feature_extraction.text import CountVectorizer

# To create a Count Vectorizer, we simply need to instantiate one.
# There are special parameters we can set here when making the vectorizer, but
# for the most basic example, it is not needed.


vectorizer = CountVectorizer()
features_CountVec = vectorizer.fit_transform(df['cleaned_text'])
print(features_CountVec.shape)
print('Sparse Matrix :\n', features_CountVec)
#features_CountVec = pd.DataFrame(features_CountVec.toarray())
features_CountVec = pd.DataFrame(features_CountVec)

features_CountVec

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


(4533, 7961)
Sparse Matrix :
   (0, 3473)	1
  (0, 1161)	1
  (0, 2340)	1
  (0, 4697)	1
  (0, 1255)	1
  (1, 4922)	1
  (1, 1976)	1
  (1, 3119)	1
  (2, 1318)	1
  (2, 4713)	1
  (2, 5381)	1
  (2, 920)	1
  (2, 2120)	1
  (2, 3333)	1
  (2, 4357)	1
  (2, 1832)	1
  (2, 1638)	1
  (3, 786)	1
  (3, 2087)	1
  (3, 2789)	1
  (3, 2070)	1
  (3, 6584)	1
  (3, 3291)	1
  (3, 7650)	1
  (3, 2010)	1
  :	:
  (4529, 3223)	1
  (4529, 7071)	1
  (4529, 992)	1
  (4530, 1756)	1
  (4530, 3504)	1
  (4530, 2308)	1
  (4530, 7780)	1
  (4530, 3397)	1
  (4530, 7408)	1
  (4530, 4053)	1
  (4530, 4354)	1
  (4531, 6584)	1
  (4531, 5430)	1
  (4531, 4825)	1
  (4531, 2338)	1
  (4531, 209)	1
  (4531, 5187)	1
  (4531, 4814)	1
  (4532, 2345)	1
  (4532, 1722)	1
  (4532, 6734)	1
  (4532, 7756)	1
  (4532, 2944)	1
  (4532, 6156)	1
  (4532, 1649)	1


,0
0,"(0, 3473)\t1\n (0, 1161)\t1\n (0, 2340)\t1..."
1,"(0, 4922)\t1\n (0, 1976)\t1\n (0, 3119)\t1"
2,"(0, 1318)\t1\n (0, 4713)\t1\n (0, 5381)\t1..."
3,"(0, 786)\t1\n (0, 2087)\t1\n (0, 2789)\t1\..."
4,"(0, 6584)\t1\n (0, 6360)\t1\n (0, 2367)\t1..."
...,...
4528,"(0, 1318)\t1\n (0, 2427)\t1\n (0, 7019)\t1..."
4529,"(0, 2461)\t1\n (0, 1756)\t1\n (0, 6889)\t1..."
4530,"(0, 1756)\t1\n (0, 3504)\t1\n (0, 2308)\t1..."
4531,"(0, 6584)\t1\n (0, 5430)\t1\n (0, 4825)\t1..."


In [50]:
# Set parameters
max_features = 10000
embedding_dim = 100
maxlen = 300
batch_size = 32
epochs = 10
# Split the dataset into training and testing sets
X = df['cleaned_text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Tokenize and pad the text sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

In [51]:
#CNN

In [52]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Sequential

# Build the CNN model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Conv1D(256, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN\t{accuracy:.2f}\t{loss:.2f}\t{auc:.2f}')


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 conv1d_4 (Conv1D)           (None, 296, 256)          128256    
                                                                 
 global_max_pooling1d_2 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_14 (Dense)            (None, 64)                16448     
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                 65        
                                                      

In [53]:
#LSTM

In [54]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Build the LSTM model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set and print the results
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'LSTM\t{accuracy:.2f}\t{loss:.2f}\t{auc:.2f}')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 lstm_5 (LSTM)               (None, 128)               117248    
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,125,569
Trainable params: 1,125,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
114/114 [========================

In [55]:
#BiLSTM

In [56]:
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Build the BiLSTM model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'BiLSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              84480     
 nal)                                                            
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,092,801
Trainable params: 1,092,801
Non-trainable params: 0
____________________________________________

In [57]:
#CNN+LSTM

In [58]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, concatenate
from tensorflow.keras.models import Sequential

# Build the CNN+LSTM model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN + LSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 300, 100)          1000000   
                                                                 
 conv1d_5 (Conv1D)           (None, 298, 128)          38528     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 149, 128)         0         
 1D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 149, 128)          0         
                                                                 
 lstm_7 (LSTM)               (None, 64)                49408     
                                                                 
 dense_20 (Dense)            (None, 64)                4160      
                                                     

In [59]:
#CNN+BiLSTM

In [60]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout, concatenate
from tensorflow.keras.models import Sequential

# Build the CNN+BiLSTM model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN + BiLSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 300, 100)          1000000   
                                                                 
 conv1d_6 (Conv1D)           (None, 298, 128)          38528     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 149, 128)         0         
 1D)                                                             
                                                                 
 dropout_14 (Dropout)        (None, 149, 128)          0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_22 (Dense)            (None, 64)              

In [61]:
# CNN
loss, accuracy, auc = model_cnn.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

# LSTM
loss, accuracy, auc = model_lstm.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'LSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

# CNN + LSTM
loss, accuracy, auc = model_cnn_lstm.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN + LSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

# BiLSTM
loss, accuracy, auc = model_bilstm.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'BiLSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

# CNN + BiLSTM
loss, accuracy, auc = model_cnn_bilstm.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN + BiLSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

NameError: name 'model_cnn' is not defined